In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 6.6MB/s 


In [0]:
from sklearn.metrics import classification_report, confusion_matrix

import pickle
import pandas as pd
import numpy as np

from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [0]:
## GLOVE EMBEDDINGS

Column_List = [ "id", "comment_text"]
Label_List = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.extend(Label_List)

# Train_Embedding_FilePath = "/content/drive/My Drive/Train_Glove_Embeddings.csv"
Train_Embedding_FilePath = "/content/drive/My Drive/cleanTrain8000_Glove_Embeddings.csv"
# Test_Embedding_FilePath = "/content/drive/My Drive/Test_Glove_Embeddings.csv"
Test_Embedding_FilePath = "/content/drive/My Drive/cleanTest2000_Glove_Embeddings.csv"

test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)
train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
train_data = train_data.fillna(1e-10)
test_data = test_data.fillna(1e-10)

X_train = train_data[Embedding_Cols]
Y_train = train_data[Label_List]
Y_train = Y_train.astype('int')

X_test = test_data[Embedding_Cols]
Y_test = test_data[Label_List]
Y_test = Y_test.astype('int')


In [5]:
classifier1 = BinaryRelevance(
    classifier = SVC(kernel="linear", probability = True),
    require_dense = [False, True],
    
)

# train
classifier1.fit(X_train, Y_train)

BinaryRelevance(classifier=SVC(C=1.0, cache_size=200, class_weight=None,
                               coef0=0.0, decision_function_shape='ovr',
                               degree=3, gamma='auto_deprecated',
                               kernel='linear', max_iter=-1, probability=True,
                               random_state=None, shrinking=True, tol=0.001,
                               verbose=False),
                require_dense=[False, True])

In [6]:
#predict
predict = classifier1.predict_proba(X_test)
predict = predict.toarray()

predict_onehot = np.empty(predict.shape)
for i in range(predict.shape[0]):
  for j in range(predict.shape[1]):
    if predict[i][j] < 0.5:
      predict_onehot[i][j] = 0
    else:
      predict_onehot[i][j] = 1

#calculate score
loss = log_loss(Y_test,predict_onehot)
print("Log_loss : {}".format(loss))
predict_onehot = np.round(predict_onehot)
loss = hamming_loss(Y_test,predict_onehot)
print("Hamming_loss : {}".format(loss*100))
accuracy = accuracy_score(Y_test,predict_onehot)
print("Accuracy : {}".format(accuracy*100))

Log_loss : 1.8749015724672382
Hamming_loss : 3.266666666666666
Accuracy : 88.25


In [7]:
classifier2 = BinaryRelevance(
    classifier = SVC(kernel="rbf",  probability = True),
    require_dense = [False, True]
)

# train
classifier2.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

BinaryRelevance(classifier=SVC(C=1.0, cache_size=200, class_weight=None,
                               coef0=0.0, decision_function_shape='ovr',
                               degree=3, gamma='auto_deprecated', kernel='rbf',
                               max_iter=-1, probability=True, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                require_dense=[False, True])

In [8]:
# predict
predict = classifier2.predict_proba(X_test)
predict = predict.toarray()

predict_onehot = np.empty(predict.shape)
for i in range(predict.shape[0]):
  for j in range(predict.shape[1]):
    if predict[i][j] < 0.5:
      predict_onehot[i][j] = 0
    else:
      predict_onehot[i][j] = 1

#calculate score
loss = log_loss(Y_test,predict_onehot)
print("Log_loss : {}".format(loss))
predict_onehot = np.round(predict_onehot)
loss = hamming_loss(Y_test,predict_onehot)
print("Hamming_loss : {}".format(loss*100))
accuracy = accuracy_score(Y_test,predict_onehot)
print("Accuracy : {}".format(accuracy*100))

Log_loss : 1.6684345038295405
Hamming_loss : 3.325
Accuracy : 87.55


In [0]:
# ## CROSS VALIDATION
# %%capture

# #define parameters for using in param grid
# parameters = [
#     {
#         'classifier': [SVC()],
#         'classifier__kernel': ['rbf', 'linear'],
#     }
# ]

# #start fitting process
# grid = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy',refit=True)
# grid_result = grid.fit(X_train, Y_train)

In [0]:
# print(grid_result)
# print('Best score : {}'.format(grid.best_score_))
# print('Best params : {}'.format(grid.best_params_))

In [0]:
# predictions = grid.predict(X_test)
# predict = grid.predict_proba(X_test)

# #calculate score
# loss = log_loss(Y_test,predict)
# print("Log_loss : {}".format(loss))
# predict = np.round(predict)
# loss = hamming_loss(Y_test,predict)
# print("Hamming_loss : {}".format(loss*100))
# accuracy = accuracy_score(Y_test,predict)
# print("Accuracy : {}".format(accuracy*100))